In [ ]:
import pandas as pd
from sklearn import model_selection
from sklearn import linear_model
from sklearn import ensemble
from sklearn import metrics
from hyperopt import fmin, hp, tpe, Trials
import numpy as np
import optuna

In [ ]:
data = pd.read_csv('_train_sem09 (1).csv')

In [ ]:
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# посмотрим на сбалансированность классов
data['Activity'].value_counts(normalize=True)

1    0.542255
0    0.457745
Name: Activity, dtype: float64

In [ ]:
X = data.drop('Activity',axis=1)
y = data['Activity']

In [ ]:
# разобьем наши данные на тренировочную и тестовую выборки
X_train,X_test,y_train,y_test = model_selection.train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
print('Размер тренировочной выборки: {}'.format(X_train.shape), '\n',
      'Размер тестовой выборки: {}'.format(X_test.shape))

Размер тренировочной выборки: (3000, 1776) 
 Размер тестовой выборки: (751, 1776)


In [ ]:
# Обучаем Логистическую регрессию с параметрами по умолчанию и смотрим целевую метрику
model = linear_model.LogisticRegression(random_state=42,max_iter=1000)
model.fit(X_train,y_train)
base_logreg_prediction = model.predict(X_test)
print('Значение f1 score с параметрами по умолчанию: {:.2f}'.format(metrics.f1_score(y_test,base_logreg_prediction)))

Значение f1 score с параметрами по умолчанию: 0.79


In [ ]:
# Обучаем случайный лес с параметрами по умолчанию и смотрим целевую метрику
model = ensemble.RandomForestClassifier()
model.fit(X_train,y_train)
base_forest_prediction = model.predict(X_test)
print('Значение f1 score с параметрами по умолчанию: {:.2f}'.format(metrics.f1_score(y_test,base_forest_prediction)))

Значение f1 score с параметрами по умолчанию: 0.83


Попробуем улучшить значение нашей метрики путем подбора гиперпараметров:

In [ ]:
model = linear_model.LogisticRegression(random_state=42,max_iter=1000)
# создадим сетку гиперпараметров, которые хотим оптимизировать
params_grid = [
    {'penalty': ['l2','none'],
    'solver': ['newton-cg','lbfgs','sag'],
    'C':[ 0.1, 0.5, 1]},
    {'penalty': ['l1','l2'],
    'solver': ['liblinear','saga'],
    'C':[ 0.1, 0.5, 1]}
]

In [ ]:
best_grid = model_selection.GridSearchCV(estimator=model,
                                        param_grid=params_grid,
                                        n_jobs=-1,
                                        cv=5,
                                        return_train_score=True)

In [ ]:
%time best_grid.fit(X_train,y_train)
grid_pred = best_grid.predict(X_test)
print('Значение f1 score с подбором параметров по сетке: {:.2f}'.format(metrics.f1_score(y_test,grid_pred)))

CPU times: user 22.4 s, sys: 3.09 s, total: 25.4 s
Wall time: 42min 46s
Значение f1 score с подбором параметров по сетке: 0.79


In [ ]:
# лучшие параметры, которые получилось подобрать 
best_grid.best_params_

{'C': 0.5, 'penalty': 'l1', 'solver': 'saga'}

после проведения подбора гиперпараметров методом GridSearch не получилось улучшить метрику. Попробуем применить Hyperopt для лучшей оптимизации

In [ ]:
 # напишем пространства с коэффициентами для оптимизации
space_1 = {
      'penalty': hp.choice('penalty',['l2','none']),
      'solver': hp.choice('solver',['newton-cg','lbfgs','sag']),
      'C':hp.quniform('C',0.1,2,0.05)}
space_2 = {
    'penalty':hp.choice('penalty',['l1','l2']),
    'solver': hp.choice('solver',['liblinear','saga']),
    'C':hp.quniform('C',0.1,2,0.05)}

trials= Trials()

# функция которую будем падавать в Hyperopt
def func(params,x=X_train,y=y_train,random_state=42):
   """
    Функция для расчета целевой метрики, для параметров из Hyperopt

    параметры
    ----------
    params : параметры из Hyperopt
    x : тренировочная выборка
    y : целевой признак тренировочной выборки
    random_state : число, необходимое для обеспечения повторяемости результатов

    """
  model = linear_model.LogisticRegression(**params,max_iter=1000,random_state=42)
  c_val = model_selection.cross_val_score(estimator=model, X=X_train,y=y_train,n_jobs=-1,scoring='f1',cv=5).mean()   # проводим кросс валидацию

  return -c_val

# производим оптимизацию с пространством space_1
hpot_1 = fmin(
    fn=func,
    space=space_1,
    trials=trials,
    max_evals=40,
    rstate=np.random.RandomState(42),
    algo=tpe.suggest
)

100%|██████████| 40/40 [42:03<00:00, 63.09s/it, best loss: -0.7864364208494703]


In [ ]:
# производим оптимизацию с пространством space_2
trials = Trials()
hpot_2 = fmin(
    fn=func,
    space=space_2,
    trials=trials,
    max_evals=40,
    rstate=np.random.RandomState(42),
    algo=tpe.suggest
)

100%|██████████| 40/40 [2:19:25<00:00, 209.15s/it, best loss: -0.7849351387765445]


In [ ]:
# имееи следующие коэффициенты
print(hpot_1,'\n',hpot_2)

{'C': 0.1, 'penalty': 0, 'solver': 1} 
 {'C': 0.45, 'penalty': 0, 'solver': 1}


In [ ]:
# Обучим модель на полученных коэффициентах и посмотрим на итоговую метрику
model = linear_model.LogisticRegression(
    penalty='l2',
    solver='lbfgs',
    C=0.1,
    max_iter=1000,
    random_state=42
    )
model.fit(X_train,y_train)
print(metrics.f1_score(y_test,model.predict(X_test)))

0.7945516458569808


In [ ]:
# Обучим модель на полученных коэффициентах и посмотрим на итоговую метрику
model = linear_model.LogisticRegression(
    penalty='l1',
    solver='saga',
    C=0.45,
    max_iter=2000,
    random_state=42
    )
model.fit(X_train,y_train)
print(metrics.f1_score(y_test,model.predict(X_test)))

0.7931428571428571


Как мы ввидим итоговая метрика после проведения оптимизации правктически не поменялась.
Попробуем обучить Случайный лес на данных и применим два метода для его оптимизации - RandomedSearchCV и Optuna

In [ ]:
# Зададим критерии поиска для рандомной оптимизации
param_distributions = {
    'criterion': ['gini','entropy'],
    'max_depth': [4,5,7,8,9,10],
    'min_samples_leaf': [3,4,5,6,7,8],
    'n_estimators': [100,120,150,170,200]
    }


In [ ]:
model = ensemble.RandomForestClassifier(random_state=42)
random_search_forest = model_selection.RandomizedSearchCV(
    param_distributions=param_distributions,
    estimator = model,
    n_jobs=-1,
    scoring='f1',
    cv=5,
    n_iter=30
)

In [ ]:
%time random_search_forest.fit(X_train,y_train)
random_search_forest_pred = random_search_forest.predict(X_test)
print('f1 score для тестовой метрики {:.2f}'.format(metrics.f1_score(y_test,random_search_forest_pred)))
print('Подобранные коэффициенты: {}'.format(random_search_forest.best_params_))

CPU times: user 4.2 s, sys: 144 ms, total: 4.35 s
Wall time: 2min 31s
f1 score для тестовой метрики 0.83
Подобранные коэффициенты: {'n_estimators': 170, 'min_samples_leaf': 3, 'max_depth': 10, 'criterion': 'entropy'}


Как мы видим итоговую метрику удалось улучшить подбором оптимальных коэффициентов

In [ ]:
# Используем Optuna и попробуем подобрать с помощью нее гиперпараметры

# напишем функцию, которую будем передавать в оптимизатор
def func(trial):
   """
    Функция для расчета целевой метрики, для параметров из Optuna

    параметры
    ----------
    trials : параметры из Optuna
    
    """
  criterion = trial.suggest_categorical('criterion',['gini','entropy'])
  max_depth = trial.suggest_int('max_depth', 4, 10, 1)
  min_samples_leaf =  trial.suggest_int('min_samples_leaf', 3, 10, 1)
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 10)

# создадим модель
  model = ensemble.RandomForestClassifier(
      n_estimators=n_estimators,
      criterion=criterion,
      max_depth=max_depth,
      min_samples_leaf=min_samples_leaf,
      random_state=42
      )
  pred = model_selection.cross_val_score(X=X_train,y=y_train,cv=5,scoring='f1',n_jobs=-1,estimator=model).mean()
  return pred
  

study = optuna.create_study(direction='maximize',study_name='RandomForestClassifier')

[I 2022-11-29 21:15:49,181] A new study created in memory with name: RandomForestClassifier


In [ ]:
study.optimize(func=func,n_jobs=-1,n_trials=40)

[I 2022-11-29 21:16:06,531] Trial 0 finished with value: 0.7747492861292312 and parameters: {'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 8, 'n_estimators': 140}. Best is trial 0 with value: 0.7747492861292312.
[I 2022-11-29 21:16:08,091] Trial 1 finished with value: 0.7903544388529932 and parameters: {'criterion': 'gini', 'max_depth': 8, 'min_samples_leaf': 5, 'n_estimators': 150}. Best is trial 1 with value: 0.7903544388529932.
[I 2022-11-29 21:16:16,611] Trial 2 finished with value: 0.7871130019484459 and parameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 8, 'n_estimators': 120}. Best is trial 1 with value: 0.7903544388529932.
[I 2022-11-29 21:16:17,489] Trial 3 finished with value: 0.7747492861292312 and parameters: {'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 8, 'n_estimators': 140}. Best is trial 1 with value: 0.7903544388529932.
[I 2022-11-29 21:16:26,294] Trial 4 finished with value: 0.7602102513223886 and parameters: {'criterion'

In [ ]:
# Выведем полученные значения параметров
print('Лучший результат на тренировочной выборке: {:.2f}'.format(study.best_value))
print('Полученные с помощью Optuna гиперпараметры: {}'.format(study.best_params))

Лучший результат на тренировочной выборке: 0.80
Полученные с помощью Optuna гиперпараметры: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 3, 'n_estimators': 150}


In [ ]:
# Обучим модель на тренировочной выборке с полученными параметрами и посмотрим метрику на тестовой выборке
model = ensemble.RandomForestClassifier(**study.best_params)
model.fit(X_train,y_train)
optuna_forest_pred = model.predict(X_test)
print('f1 score на тестовой выборке: {:.2f}'.format(metrics.f1_score(y_test,optuna_forest_pred)))

f1 score на тестовой выборке: 0.82
